In [25]:
import os
import extcolors
from PIL import Image
from pdf2image import convert_from_path, convert_from_bytes
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import easyocr

#### functions

In [26]:
def get_colors(img_path):
    image = convert_from_path(img_path)
    img = image[0].convert("RGB")
    return extcolors.extract_from_image(img, tolerance=33, limit=10)

def create_image_entry(colors, file_name):
    total_pixels = colors[1]
    image_dict = {'file_name': file_name}
    image_colors = [{'colorCode': str(index), 'percent': round(color / total_pixels * 100)} for index, color in
                    colors[0] if round(color / total_pixels * 100) >= 1]
    image_dict['img_colors'] = image_colors

    return image_dict

#### PDF to JPG 

In [27]:
directory = 'PDFs'
for dirpath, _, filenames in os.walk(directory):
    filenames_to_process = len(filenames)
    for f in filenames:
        img_path = os.path.abspath(os.path.join(dirpath, f))
        filename = os.path.basename(f)[:-4]
        pages = convert_from_path(img_path)
        new_path =  f"JPGs/{filename}"
        for i, page in enumerate(pages):
            if not os.path.exists(new_path):
                os.makedirs(new_path)
            path = f"{new_path}/{filename}_{i}.jpg"
            page.save(path, "JPEG")

KeyboardInterrupt: 

In [ ]:
### TOHLE JEŠTĚ PŘEDĚLAT NA JPG
directory = "PDFs"
collection = []
for dirpath, _, filenames in os.walk(directory):
    filenames_to_process = len(filenames)
    for f in filenames:
        img_path = os.path.abspath(os.path.join(dirpath, f))
        filename = os.path.basename(f)
        colors = get_colors(img_path)
        print(f'{filename}: {colors}')
        file_dict = create_image_entry(colors, filename)
        collection.append(file_dict)


PDFPageCountError: Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table


#### RBG distribution 

In [ ]:
directory = 'JPGs'
for dirpath, _, filenames in os.walk(directory):
    filenames_to_process = len(filenames)
    for f in filenames: 
        img_path = os.path.abspath(os.path.join(dirpath, f))
        filename = os.path.basename(f)
        img = cv2.imread(path)
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        colors = ('r', 'g', 'b')
        for i, col in enumerate(colors):
            hist = cv2.calcHist([image], [i], None, [256], [0, 256])
            plt.plot(hist, color=col)
            plt.show()

#### Saturation 

In [ ]:
directory = 'JPGs'
for dirpath, _, filenames in os.walk(directory):
    filenames_to_process = len(filenames)
    for f in filenames: 
        img_path = os.path.abspath(os.path.join(dirpath, f))
        filename = os.path.basename(f)
        img = cv2.imread(img_path)
        hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        saturation = hsv[:, :, 1]
        vibrancy = saturation.mean()
        print(f"{filename} Average saturation (vibrancy): {vibrancy}")

The Tricky Questions of Smartie Joey_15.jpg Average saturation (vibrancy): 86.70452707350306
The Tricky Questions of Smartie Joey_14.jpg Average saturation (vibrancy): 15.708740016385613
The Tricky Questions of Smartie Joey_16.jpg Average saturation (vibrancy): 3.6123088631901377
The Tricky Questions of Smartie Joey_17.jpg Average saturation (vibrancy): 31.70985333686331
The Tricky Questions of Smartie Joey_13.jpg Average saturation (vibrancy): 68.1594857245502
The Tricky Questions of Smartie Joey_12.jpg Average saturation (vibrancy): 18.861040229874266
The Tricky Questions of Smartie Joey_10.jpg Average saturation (vibrancy): 28.136898737781983
The Tricky Questions of Smartie Joey_11.jpg Average saturation (vibrancy): 71.5187456305551
The Tricky Questions of Smartie Joey_8.jpg Average saturation (vibrancy): 25.82402539081692
The Tricky Questions of Smartie Joey_9.jpg Average saturation (vibrancy): 65.7684916785741
The Tricky Questions of Smartie Joey_1.jpg Average saturation (vibrancy

### Dominant colours

In [ ]:
directory = 'JPGs'
for dirpath, _, filenames in os.walk(directory):
    filenames_to_process = len(filenames)
    for f in filenames[1:]: 
        img_path = os.path.abspath(os.path.join(dirpath, f))
        filename = os.path.basename(f)
        img = cv2.imread(img_path)
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pixels = image.reshape(-1, 3)
        kmeans = KMeans(n_clusters=5).fit(pixels)
        dominant_colors = kmeans.cluster_centers_.astype(int)
        print("Dominant colors:", dominant_colors)

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/numpy/core/multiarray.py", line 346, in where
    @array_function_from_c_func_and_dispatcher(_multiarray_umath.where)
    
KeyboardInterrupt: 


Dominant colors: [[244 246 244]
 [211 218 210]
 [ 60  54  52]
 [240 196 134]
 [150 176 184]]


/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Dominant colors: [[165 205 236]
 [253 251 251]
 [ 28  29  29]
 [230 164 143]
 [120 102  80]]


/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
